# Application Demo
Demonstrates how VMC might be used in conjunction with application-specific data

---

In [20]:
import json

from support.vmchelper import VMCHelper 
from support.utils import json_pretty_format

def opp(o):
    """object pretty print -- prints any VMC object"""
    print(json_pretty_format(o.serialize()))

In [29]:
class App:
    def __init__(self):
        self.vmcdata = VMCHelper()
        self.annotations = {}

    def as_json(self):
        return json.dumps({
            "appinfo": {"what": "ever"},
            "vmcbundle": self.vmcdata.as_bundle(),
            "annotations": self.annotations
        })
    
    def get_record(self, id):
        """given variation id, return dictionary of all associated data"""
        return {
            "type": "Allele",
            "object": "Allele(...)",
            "pop_freq": "{CEU: }",
            "ann key": "value from id lookup in annotations"
        }
    
app = App()

In [26]:
# *1: https://www.pharmgkb.org/haplotype/PA165819268
# *3A: https://www.pharmgkb.org/haplotype/PA165819270
# *3B: https://www.pharmgkb.org/haplotype/PA165819271
# *3C: https://www.pharmgkb.org/haplotype/PA165819272

# TPMT_hgvs_alleles
rs1142345T = "NC_000006.11:g.18130918="
rs1142345C = "NC_000006.11:g.18130918T>C"
rs1800460C = "NC_000006.11:g.18139228="
rs1800460T = "NC_000006.11:g.18139228C>T"

TPMT_hgvs_haplotypes = {
    "TPMT*1": [rs1142345T, rs1800460C],
    "TPMT*3A": [rs1142345C, rs1800460T],
    "TPMT*3B": [rs1800460T],
    "TPMT*3C": [rs1142345C],
}

# Data from CPIC via Bob Freimuth (thanks!)
pop_labels = "CEU MED SA AFR".split()
pop_freq = {
    "TPMT*1": dict(zip(pop_labels, [0.95726, 0.96081, 0.95233, 0.93901])),
    "TPMT*3A": dict(zip(pop_labels, [0.0356, 0.0254, 0.0287, 0.00198])),
    "TPMT*3B": dict(zip(pop_labels, [0.000561, 0.00426, 0.000486, 0])),
    "TPMT*3C": dict(zip(pop_labels, [0.004205, 00.00545, 0.00924, 0.0495])),
}

In [27]:
app.annotations["pop_freq"] = {}
for hapname in "TPMT*1 TPMT*3A TPMT*3B TPMT*3C".split():
    haplotype = app.vmcdata.add_hgvs_haplotype(TPMT_hgvs_haplotypes[hapname])
    app.vmcdata.identifiers[haplotype.id].add(hapname)
    app.annotations["pop_freq"][haplotype.id] = pop_freq[hapname]

In [28]:
app.as_json()

'{"appinfo": {"what": "ever"}, "annotations": {"pop_freq": {"VMC:GH_BsOxQ7mEbHhNXNkpiDrEHzFbYQ9oA4dB": {"CEU": 0.95726, "MED": 0.96081, "SA": 0.95233, "AFR": 0.93901}, "VMC:GH_kxTWlh02KybqQybArmci5ScJbwNcTNlC": {"CEU": 0.0356, "MED": 0.0254, "SA": 0.0287, "AFR": 0.00198}, "VMC:GH_Pi0KRfcBXsMlBW2th1tBNul53qJ5P_TM": {"CEU": 0.000561, "MED": 0.00426, "SA": 0.000486, "AFR": 0}, "VMC:GH_EepV7kZVXZ-FfWA7lRgvVTdxQVMS8ozR": {"CEU": 0.004205, "MED": 0.00545, "SA": 0.00924, "AFR": 0.0495}}}}'